# WB-XIC, Lab2: Wstęp do sieci neuronowych i PyTorch

Wymagania: Python, NumPy, Jupyter Notebook

Bazując na [Dive into Deep Learning](https://d2l.ai/index.html)

PyTorch:
- https://pytorch.org
- https://github.com/pytorch/pytorch

## Wstęp

In [1]:
# pip install torch
import torch
import numpy as np

In [2]:
# np.arange(10, dtype=np.float32)
x = torch.arange(10, dtype=torch.float32)
x

tensor([0., 1., 2., 3., 4., 5., 6., 7., 8., 9.])

In [3]:
x.shape

torch.Size([10])

In [4]:
x.numel()

10

In [5]:
x.reshape(5, 2)

tensor([[0., 1.],
        [2., 3.],
        [4., 5.],
        [6., 7.],
        [8., 9.]])

In [6]:
x.reshape(5, 2).numel()

10

In [7]:
torch.zeros((1, 2))

tensor([[0., 0.]])

In [8]:
torch.ones((2, 3, 4))

tensor([[[1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.]],

        [[1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.]]])

In [9]:
torch.randn(5)

tensor([ 0.0709,  0.1597, -0.1381, -2.0224,  0.2959])

In [10]:
torch.randint(10, 5) # ?

TypeError: randint(): argument 'size' (position 2) must be tuple of ints, not int

In [11]:
# 1.
a = torch.tensor([1, 2, 3])
# 2.
b = torch.from_numpy(np.array([2, 3, 4]))

In [12]:
a + b 

tensor([3, 5, 7])

### Zadanie 1.
1. stworzyć poziomy wektor `x` z 4 unikalnymi wartościami całkowitymi
2. stworzyć macierz `X` składającą się z wierszy: wszystkich możliwych permutacji wektora `x`
3. stworzyć pionowy wektor `w` z 4 wartościami z rozkładu normalnego
4. pomnożyć macierzowo `X*w`
5. znaleźć indeksy dla maksimum i minimum wierszy

In [13]:
torch.manual_seed(7)
np.random.seed(7)

## [Pochodna i różniczkowanie](https://d2l.ai/chapter_preliminaries/calculus.html#derivatives-and-differentiation)

In [23]:
def f(x):
    return x ** 2 - 2 * x

In [24]:
def numerical_lim(f, x, h):
    return (f(x + h) - f(x)) / h

In [25]:
h = 1
for i in range(5):
    print(f'x = {1}, h={h:.5f}, numerical limit={numerical_lim(f, 1, h):.5f}')
    h *= 0.1

x = 1, h=1.00000, numerical limit=1.00000
x = 1, h=0.10000, numerical limit=0.10000
x = 1, h=0.01000, numerical limit=0.01000
x = 1, h=0.00100, numerical limit=0.00100
x = 1, h=0.00010, numerical limit=0.00010


Wynik: https://www.wolframalpha.com/input?i=x+**+2+-+2+*+x

### Zadanie 2.

`y_hat = torch.mm(X, w)` (z Zadania 1.) 
1. stworzyć pionowy wektor `y` poprzez dodanie szumu z rozkładu normalnego do `y_hat`
2. policzyć średni błąd kwadratowy [`MSE(y, y_hat)`](https://pl.wikipedia.org/wiki/B%C5%82%C4%85d_%C5%9Bredniokwadratowy)
3. napisać wzór na pochodną `MSE` względem `w` (teoretycznie, w markdown)
4. policzyć pochodną `MSE` po `w`

**Uwaga!** Wyraz wolny, bias - `b` -> [szczegóły](https://d2l.ai/chapter_linear-networks/linear-regression.html)

## Gradient descent na przykładzie regresji liniowej

### Zadanie 3.

Spróbuj zaimplementować poniższy pseudokod.

In [31]:
learning_rate = 0.001
epochs = 100
for i in range(epochs):
    # 1. policz gradient MSE
    # 2. zaktualizuj wagi wykorzystując gradient i learning_rate
    pass
# 3. policz MSE

**Uwaga!** Wyraz wolny, bias - `b` -> [szczegóły](https://d2l.ai/chapter_linear-networks/linear-regression.html)

## Klasyfikacja

Główna różnica w klasyfikacji to dodanie nowego wymiaru - klasy (np. `3` dla danych iris, `10` dla MNIST):
1. wektor `y` w klasyfikacji binarnej ma dwa wymiary `(N, 2)`
2. wektor `y_hat` musi być normalizowany - prawdopodobieństwa przynależności do klas sumują się do 1 wykorzystując [SoftMax](https://d2l.ai/chapter_linear-networks/softmax-regression-concise.html#softmax-implementation-revisited)
3. stosujemy funkcję straty Cross entropy zamiast MSE; zmienia się sposób liczenia pochodnej

In [38]:
torch.manual_seed(7)
np.random.seed(7)

In [39]:
num_inputs = 3
X = np.random.normal(size=(100, num_inputs))
y = (X.sum(axis=1) > 0).astype(np.int)
Y = np.column_stack((y, 1- y))
num_outputs = Y.shape[1]
X = torch.as_tensor(X, dtype=torch.float32)
Y = torch.as_tensor(Y, dtype=torch.float32)

In [40]:
# requires_grad - informujemy o zaalokowaniu dodatkowej pamięci na gradient
W = torch.normal(0, 0.01, size=(num_inputs, num_outputs), requires_grad=True)
b = torch.zeros(num_outputs, requires_grad=True)

In [41]:
def softmax(X):
    X_exp = torch.exp(X)
    partition = X_exp.sum(1, keepdim=True)
    return X_exp / partition

In [42]:
def neural_network(X):
    # w tym przypadku architektura odpowiada regresji logistycznej
    return softmax(torch.mm(X.reshape((-1, W.shape[0])), W) + b)

In [43]:
def cross_entropy(y_hat, y):
    return - torch.log(y_hat[range(len(y_hat)), y])

In [44]:
optimizer = torch.optim.SGD({W, b}, lr=0.001)
errors = []

for i in range(100):
    y_hat = neural_network(X)
    errors += [np.abs((torch.as_tensor(y) - y_hat.argmax(1)).numpy()).mean()]
    l = cross_entropy(y_hat, y)
    optimizer.zero_grad()
    l.mean().backward()
    optimizer.step()

In [46]:
torch.as_tensor(y) - y_hat.argmax(1)

tensor([ 0,  0,  0, -1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  1,  0,  0,
         0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,
         0,  0,  0,  0,  0, -1,  1, -1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, -1,  0, -1,  0,  0,
         0,  0,  0,  0,  0,  0,  1,  1,  0,  0])

In [47]:
W

tensor([[-0.0154,  0.0218],
        [-0.0107,  0.0090],
        [-0.0087,  0.0167]], requires_grad=True)

In [48]:
b

tensor([ 2.5413e-06, -2.5412e-06], requires_grad=True)

*policzyć macierz pomyłek klasyfikacji binarnej

# Praca Domowa 1

Indywidualnie, zaimplementować model sieci neuronowej (MLP) do klasyfikacji w `torch` i przetestować go na dwóch zbiorach danych: XOR (wygenerować), oraz iris (pobrać). Powstały raport w formatach `.ipynb` oraz `.html` powinien zawierać wionski z przeprowadzonej analizy.
1. do 2 punktów uzyskuje się za skuteczność klasyfikacji modeli i wizualizację procesu uczenia
2. do 2 punktów uzyskuje się za analizę porównawczą różnych architektur sieci neuronowych (liczby neuronów i warstw)
3. 1 punkt uzyskuje się za analizę porównawczą różnych wartości `learning_rate` 
4. 1 punkt uzyskuje się za wytrenowanie skutecznego modelu na zbiorze danych MNIST i analizę macierzy pomyłek tej klasyfikacji
5. 1 punkt uzyskuje się za ewaluowanie powyższych zjawisk na podzbiorze treningowym i testowym (analizę zjawiska przeuczenia)
6. 1 punkt uzyskuje się za animację zmiany granic decyzyjnych klasyfikacji podczas uczenia (na wybranym zbiorze danych)
7. do 2 punktów uzyskuje sie za jakość raportu (opisu, wizualizacji, kodu).

**Uwaga!** Warto zacząć od uczenia bardzo małych sieci i stopniowo zwiększać ich skomplikowanie.

Deadline: 16 marca 23:59. Na zajęciach 17 marca 5 wybranch osób krótko zaprezentuje swoje wyniki.